$$h_{N(v)} = \sum_{j \in N(v)} \frac{1}{\sqrt[]{d_{v}+1}\cdot \sqrt[]{d_{j}+1}}h_{j}$$

$\sqrt[]{d_{v}+1}$

In [1]:
import os
# import Google Drive 套件
from google.colab import drive
# 將自己的雲端硬碟掛載上去
drive.mount('/content/gdrive')

os.chdir('./gdrive/MyDrive/Colab Notebooks/cheng_ta/hw1')      # 檔案目錄

Mounted at /content/gdrive


In [2]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [3]:
import importlib
import random
import pickle
import logging
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import os
import time
import fun_cls as fc
import DrBC_modle as Dm
import function as func
from sklearn.model_selection import train_test_split

# load test data

In [4]:
path = './test_data/syn_data/20000/'
files = os.listdir(path)

In [5]:
testDatas = []
# for i in files:
#     with open(path + i, 'rb') as f:
#         G = pickle.load(f)
#         testDatas.append(G)
for i in range(len(files)):
  testDatas.append(path + files[i])

# load model

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [9]:
drbcModule = Dm.DrBC_module(Layer=5, device=device).to(device)
drbcModule.training = False
drbcModule.training

False

In [11]:
drbcModule.load_state_dict(torch.load('./modules/DrBC_5Layer_10000.pt', map_location=device))

<All keys matched successfully>

# start test

In [ ]:
# test time and test result
testResult = []
timeResult = []
realPred = []
for i, j in enumerate(testDatas):
    with open(j, 'rb') as f:
      G = pickle.load(f)
    t = time.time()
    output = drbcModule(G).T
    t = time.time() - t
    timeResult.append(t)
    testResult.append(output.clone().detach().cpu().numpy())
    realPred.append(G.scoreList)
    del(G)
timeResult = np.array(timeResult)
print(np.mean(timeResult))
print(np.std(timeResult))

In [18]:
# t = time.time()
# testResult = []
# for i, j in enumerate(testDatas):
#     with open(j, 'rb') as f:
#       G = pickle.load(f)
#     output = drbcModule(G).T
#     testResult.append(output.clone().detach().cpu().numpy())
# # with open('./test_data/syn_test_result/sys_result_5L_new.pickle', 'wb') as f:
# #     pickle.dump(testResult, f)
# t = time.time() - t
# print(t / len(testDatas))

1.8515127182006836


In [ ]:
# with open('./test_data/syn_test_result/sys_result.pickle', 'rb') as f:
#     testResult = pickle.load(f)

In [ ]:
importlib.reload(func)

<module 'function' from 'D:\\study\\成大\\soa2\\cheng_te\\hw1\\function.py'>

In [41]:
realPred = []
for i, j in enumerate(testDatas):
  with open(j, 'rb') as f:
    G = pickle.load(f)
    realPred.append(G.scoreList)

In [44]:
# top n test
N = 5000
n = int(N * 0.1)
accuracy = []
for i, j in enumerate(testResult):
    pred = j[0, :]
    truths = np.array(realPred[i])
    accuracy.append(func.top_n_accuracy(pred, truths, n=n))
accuracy, np.mean(np.array(accuracy)), np.std(np.array(accuracy))

([0.944,
  0.942,
  0.946,
  0.944,
  0.936,
  0.952,
  0.94,
  0.942,
  0.93,
  0.938,
  0.942,
  0.954,
  0.938,
  0.93,
  0.936,
  0.956,
  0.934,
  0.948,
  0.954,
  0.938,
  0.946,
  0.944,
  0.958,
  0.946,
  0.94,
  0.942,
  0.942,
  0.956,
  0.94,
  0.944],
 0.9434,
 0.007213875518748554)

In [45]:
# Kendall tau distance
tau_distance = []
for i, j in enumerate(testResult):
    pred = j[0, :]
    truths = np.array(realPred[i])
    tau_distance.append(func.normalised_kendall_tau_distance(pred, truths))
tau_distance, np.mean(np.array(tau_distance)), np.std(np.array(tau_distance))

([0.8993423542354235,
  0.8973811781178118,
  0.9011099509950995,
  0.897439403940394,
  0.8989323732373238,
  0.9010185018501851,
  0.898505810581058,
  0.8993248724872487,
  0.8985176717671767,
  0.8960997499749975,
  0.8977183918391839,
  0.8999154515451545,
  0.9006143614361436,
  0.8982460246024603,
  0.8948824882488249,
  0.8974063806380638,
  0.8983415541554155,
  0.8995798779877988,
  0.8977193319331933,
  0.8958120212021202,
  0.8972653665366537,
  0.8960695869586959,
  0.899173697369737,
  0.8968063606360636,
  0.8971276727672767,
  0.8994085008500851,
  0.9002077007700771,
  0.8977179117911791,
  0.8968591859185918,
  0.8974906290629063],
 0.8982011454478782,
 0.001542176924545882)

# try

In [ ]:
c = 0
pred = testResult[c][0, :]
truths = np.array(testDatas[c].scoreList)
accuracy = func.top_n_accuracy(pred, truths, n=5)
accuracy

0.8

In [ ]:
np.argsort(pred)[-5:], np.argsort(truths)[-5:]

(array([2, 1, 4, 5, 0]), array([6, 1, 4, 5, 0]))

In [ ]:
np.argmax(testResult[c])

0

In [ ]:
accuracy

1.0

In [ ]:
np.argsort(pred)

array([3897, 3813, 2743, ...,    4,    5,    0])

In [ ]:
np.argsort(np.array([6, 3, 5]))

array([1, 2, 0])

In [ ]:
pred[3897], pred[0]

(-0.085591435, 0.009903476)

In [ ]:
np.argmax(np.array([6, 3, 5]))

0

In [ ]:
np.linspace(0, 1, 5)

array([0.  , 0.25, 0.5 , 0.75, 1.  ])

In [ ]:
n = 5
a, b = np.meshgrid(np.arange(n), np.arange(n))
a, b

(array([[0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4]]), array([[0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2],
        [3, 3, 3, 3, 3],
        [4, 4, 4, 4, 4]]))

In [ ]:
c = 5
values1 = testResult[c][0, :]
values2 = np.array(testDatas[c].scoreList)
n = len(values1)
if n != len(values2):
    print('wrong length')
i, j = np.meshgrid(np.arange(n), np.arange(n))
ndisordered = np.logical_or(np.logical_and(values1[i] <= values1[j], values2[i] <= values2[j]), np.logical_and(values1[i] >= values1[j], values2[i] >= values2[j])).sum()
ndisordered / (n * (n - 1))

0.7838455691138227

In [ ]:
values1

array([0.29478282, 0.2649094 , 0.24649474, ..., 0.19568603, 0.19565482,
       0.19564813], dtype=float32)

In [ ]:
a

array([4483, 4683, 1171, ...,    4,    5,    0])

In [ ]:
np.logical_and(values1[i] < values1[j], values2[i] < values2[j])

array([[False,  True,  True, ...,  True,  True,  True],
       [False, False,  True, ...,  True,  True,  True],
       [False, False, False, ...,  True,  True,  True],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [ ]:
values1[i]

array([[0.29478282, 0.2649094 , 0.24649474, ..., 0.19568603, 0.19565482,
        0.19564813],
       [0.29478282, 0.2649094 , 0.24649474, ..., 0.19568603, 0.19565482,
        0.19564813],
       [0.29478282, 0.2649094 , 0.24649474, ..., 0.19568603, 0.19565482,
        0.19564813],
       ...,
       [0.29478282, 0.2649094 , 0.24649474, ..., 0.19568603, 0.19565482,
        0.19564813],
       [0.29478282, 0.2649094 , 0.24649474, ..., 0.19568603, 0.19565482,
        0.19564813],
       [0.29478282, 0.2649094 , 0.24649474, ..., 0.19568603, 0.19565482,
        0.19564813]], dtype=float32)

In [ ]:
values1[1]

0.2649094